In [1]:
%%capture
## !apt-get update
!apt install firefox
!pip install selenium

In [42]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import time

import pandas as pd
import numpy as np
import datetime as dt

import sys


In [43]:
# Set up Firefox options
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15,gzip(gfe)'

options = webdriver.FirefoxOptions()
options.add_argument('--headless')
options.add_argument(f'user-agent={user_agent}')

# Set up desired capabilities
capabilities = DesiredCapabilities.FIREFOX.copy()
capabilities['acceptInsecureCerts'] = True

## some explanations of Firefox profile here: https://www.browserstack.com/guide/download-file-using-selenium-python
## and here: https://www.browserstack.com/docs/automate/selenium/firefox-profile#python
## official Mozilla documentation for Firefox profile: https://support.mozilla.org/en-US/kb/profile-manager-create-remove-switch-firefox-profiles
profile = FirefoxProfile()
profile.set_preference('browser.download.folderList', 2) # Save downloads to the desktop
profile.set_preference('browser.download.manager.showWhenStarting', False) # Don't show the download manager
profile.set_preference('browser.download.dir', '/content') # Set the download directory
profile.set_preference('browser.helperApps.neverAsk.saveToDisk', 'application/zip') # Set the MIME type of the file to download

# Initialize Firefox driver with options and capabilities
driver = webdriver.Firefox(options=options, capabilities=capabilities, firefox_profile=profile)



<ipython-input-43-e5dfd6b4dac8>:15: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  profile = FirefoxProfile()
<ipython-input-43-e5dfd6b4dac8>:22: DeprecationWarning: capabilities and desired_capabilities have been deprecated, please pass in a Service object
  driver = webdriver.Firefox(options=options, capabilities=capabilities, firefox_profile=profile)
<ipython-input-43-e5dfd6b4dac8>:22: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  driver = webdriver.Firefox(options=options, capabilities=capabilities, firefox_profile=profile)


In [44]:

driver.get("https://www.facebook.com/ads/library/report/?source=archive-landing-page&country=US")
time.sleep(15)
driver.save_screenshot("first_screen.png")

tabs = ['All dates', 'Last 90 days', 'Last 30 days', 'Last 7 days' , 'Last day']
dl_folders = ['', '', '', '', '']

dl_df = pd.DataFrame({'tab': tabs, 'folder': dl_folders})

## scroll to the bottom of the page
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
time.sleep(10)

## webdriver.ActionChains(driver).send_keys(Keys.CONTROL, Keys.END).perform()

## find the drop-down list

driver.save_screenshot("second_screen.png")
tab = driver.find_elements(by=By.XPATH, value = "//div[contains(text(), 'Time range')]/following-sibling::div")

if len(tab) == 0:
  print("The time range element not found")


In [45]:
for k in range(dl_df.shape[0]):
  xpath_tab = dl_df['tab'][k]
  print(xpath_tab)
  ## open the menu
  webdriver.ActionChains(driver).move_to_element(tab[0]).perform()
  time.sleep(2)
  webdriver.ActionChains(driver).click(tab[0]).perform()
  time.sleep(5)
  driver.save_screenshot("third_screen.png")

  ## select the item
  ## menu_item = driver.find_elements(by=By.XPATH, value = "//div[contains(@role, 'menu')]/descendant::div[contains(text(), 'All dates')]")
  menu_item = driver.find_elements(by=By.XPATH, value = "//div[contains(@role, 'menu')]/descendant::div[contains(text(), '{}')]".format(xpath_tab) )
  if len(menu_item) == 0:
    print("menu item not found")
    break

  webdriver.ActionChains(driver).move_to_element(menu_item[0]).perform()
  webdriver.ActionChains(driver).click(menu_item[0]).perform()  
  webdriver.ActionChains(driver).reset_actions()
  driver.save_screenshot("fourth_screen.png")
  #continue


  webdriver.ActionChains(driver).reset_actions()

  try:
      ## locate the download button
      button = driver.find_elements(by = By.XPATH, value="//div[contains(translate(text(), 'DR', 'dr'), 'download report')]/parent::a")
      ## second_button = button[1]
      if len(button) == 1:
        second_button = button[0]
      else:
        second_button = button[1]

      ## move the focus onto the button
      webdriver.ActionChains(driver).move_to_element(second_button).perform()
      time.sleep(2)

      ## click on the button to initiate the download
      webdriver.ActionChains(driver).click(second_button).perform()
      time.sleep(15)

  except NoSuchElementException:
      print('Download report button not found for' + xpath_tab + ' tab')
      ## driver.close()
      ## driver.quit()


All dates
Last 90 days
Last 30 days
Last 7 days
Last day


In [46]:
driver.close()
driver.quit()

In [ ]:
webdriver.ActionChains(driver).move_to_element(tab[0]).perform()
time.sleep(2)

## click on the tab
webdriver.ActionChains(driver).click(tab[0]).perform()
time.sleep(5)
driver.save_screenshot("third_screen.png")


True

In [ ]:
menu_item = driver.find_elements(by=By.XPATH, value = "//div[contains(@role, 'menu')]/descendant::div[contains(text(), 'All dates')]")
for m in menu_item:
  print(m.location)
  print(m.size)
  print(m.is_displayed())